<a href="https://colab.research.google.com/github/daniaguizap/BigQuery/blob/main/Upload_Data_to_BigQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Upload the data to Colab**

In [ ]:
import pandas as pd

In [ ]:
df_costs = pd.read_csv("/content/costs.csv")

df_costs.head()

,country,whatsapp_usd,sms_usd
0,AC,0.043,0.107000
1,AD,0.043,0.130000
2,AE,0.026,0.063066
3,AF,0.040,0.180605
4,AG,0.043,0.108000


In [ ]:
df_profiles = pd.read_csv("/content/profiles.csv")

df_profiles.head()

,userID,gender,dob,country
0,mpr_2fOzSxcEMql0TQkeK2vGCuJBu9K,M,"25 Oct, 2005, 00:00",CA
1,mpr_2fIcL4JraEe6OMdD4xvfVIZiAx8,M,"10 Jun, 1978, 00:00",CA
2,mpr_2fHMFzUtfSXQORtB5anUIjuNQso,F,"15 Feb, 2002, 00:00",GB
3,mpr_2fHaKh68oUhLyEZwztVzaCHefQ9,F,"25 Aug, 2001, 00:00",FR
4,mpr_2fIJtYAsOQ8AqU402d8j5yEFLqt,M,"18 Apr, 1990, 00:00",FR


In [ ]:
df_verification = pd.read_csv("/content/verification.csv")

df_verification.head()

,userID,group,method,verified
0,mpr_2fHOBr0Nw6mNM2fPIKMeSyKUP0e,B,Sms,0
1,mpr_2fHOmoydcZfjF0UdFEmDQQifYK1,C,Sms,1
2,mpr_2fHRBhLrWEOcfjjblwatVXbnxoN,B,Sms,1
3,mpr_2fHQGGdXwbv8902hxC9C1dt1ThN,A,Sms,1
4,mpr_2fHSJ6Yyz6VC06GVQFjXH9q7s6m,A,Sms,1


# **Nulls and Duplicates verification**

In [ ]:
df_costs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228 entries, 0 to 227
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country       228 non-null    object 
 1   whatsapp_usd  228 non-null    float64
 2   sms_usd       228 non-null    float64
dtypes: float64(2), object(1)
memory usage: 5.5+ KB


In [ ]:
df_costs.isnull().sum()

,0
country,0
whatsapp_usd,0
sms_usd,0


In [ ]:
df_costs.duplicated().value_counts()

,count
False,228


In [ ]:
df_profiles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18424 entries, 0 to 18423
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   userID   18424 non-null  object
 1   gender   18424 non-null  object
 2   dob      18424 non-null  object
 3   country  18421 non-null  object
dtypes: object(4)
memory usage: 575.9+ KB


In [ ]:
df_profiles.isnull().sum()

,0
userID,0
gender,0
dob,0
country,3


In [ ]:
df_profiles.duplicated().value_counts()

,count
False,18424


In [ ]:
df_verification.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18424 entries, 0 to 18423
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   userID    18424 non-null  object
 1   group     18424 non-null  object
 2   method    18424 non-null  object
 3   verified  18424 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 575.9+ KB


In [ ]:
df_verification.isnull().sum()

,0
userID,0
group,0
method,0
verified,0


In [ ]:
df_verification.duplicated().value_counts()

,count
False,18424


# **Authenticate and access Google Cloud services**

In [ ]:
from google.colab import auth
auth.authenticate_user()

# **Load data into BigQuery**

**1. Create a new dataset in your BigQuery to store your project data**

In [ ]:
from google.cloud import storage
from google.cloud import bigquery

In [ ]:
def create_dataset(project_id, dataset_id, location="US"):
    client = bigquery.Client(project=project_id)
    dataset_ref = bigquery.DatasetReference(project_id, dataset_id)
    dataset = bigquery.Dataset(dataset_ref)
    dataset.location = location

    created_dataset = client.create_dataset(dataset, timeout=30)  # Waits 30 seconds
    print("Dataset {} created.".format(created_dataset.dataset_id))

# Usage
create_dataset('muzz-440719', 'Muzzab1') # Replace 'your-project-id' with your Google Cloud project ID and 'your-new-datatset-name' with your desired dataset name

**2. Load data into the dataset you just created in BigQuery**

In [ ]:
import pandas_gbq

project_id = "muzz-440719"
dataset_name = "Muzzab1"
table_name = "Costs"

df_final = df_costs #Your DataFrame

# Define your BigQuery client
bigquery_client = bigquery.Client(project=project_id)

# Convert TableReference to table name string
table_ref_str = f"{project_id}.{dataset_name}.{table_name}"

# Use Pandas to upload DataFrame to BigQuery
df_final.to_gbq(destination_table=table_ref_str, project_id=project_id, if_exists="replace")

print(f"Data loaded into BigQuery: {project_id}.{dataset_name}.{table_name}")

<ipython-input-10-d45a753fc4ec>:16: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df_final.to_gbq(destination_table=table_ref_str, project_id=project_id, if_exists="replace")
100%|██████████| 1/1 [00:00<00:00, 745.79it/s]

Data loaded into BigQuery: muzz-440719.Muzzab1.Costs


In [ ]:
import pandas_gbq

project_id = "muzz-440719"
dataset_name = "Muzzab1"
table_name = "Profiles"

df_final2 = df_profiles #Your DataFrame

# Define your BigQuery client
bigquery_client = bigquery.Client(project=project_id)

# Convert TableReference to table name string
table_ref_str = f"{project_id}.{dataset_name}.{table_name}"

# Use Pandas to upload DataFrame to BigQuery
df_final2.to_gbq(destination_table=table_ref_str, project_id=project_id, if_exists="replace")

print(f"Data loaded into BigQuery: {project_id}.{dataset_name}.{table_name}")

<ipython-input-11-767d58efa918>:16: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df_final2.to_gbq(destination_table=table_ref_str, project_id=project_id, if_exists="replace")
100%|██████████| 1/1 [00:00<00:00, 802.43it/s]

Data loaded into BigQuery: muzz-440719.Muzzab1.Profiles


In [ ]:
import pandas_gbq

project_id = "muzz-440719"
dataset_name = "Muzzab1"
table_name = "Verification"

df_final3 = df_verification #Your DataFrame

# Define your BigQuery client
bigquery_client = bigquery.Client(project=project_id)

# Convert TableReference to table name string
table_ref_str = f"{project_id}.{dataset_name}.{table_name}"

# Use Pandas to upload DataFrame to BigQuery
df_final3.to_gbq(destination_table=table_ref_str, project_id=project_id, if_exists="replace")

print(f"Data loaded into BigQuery: {project_id}.{dataset_name}.{table_name}")

<ipython-input-12-f7485332bcd7>:16: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df_final3.to_gbq(destination_table=table_ref_str, project_id=project_id, if_exists="replace")
100%|██████████| 1/1 [00:00<00:00, 1405.13it/s]

Data loaded into BigQuery: muzz-440719.Muzzab1.Verification


# **Extract data from Big Query**


In [ ]:
# Import the necessary libraries
from google.cloud import storage
from google.cloud import bigquery

# First make connection to BigQuery
# Replace 'your-project-id' with your Google Cloud project ID
# Replace 'your-dataset-name' with your dataset name
project_id = "muzz-440719"
dataset_name = "Muzzab1"
table_name = "DataFinal1" # if you make changes previously, then here you need to make according change

bigquery_client = bigquery.Client(project=project_id)

In [ ]:
# Query all data from the table
query = f"""
    SELECT *
    FROM `{project_id}.{dataset_name}.{table_name}`
"""

query_job = bigquery_client.query(query)
df_final1 = query_job.result()

df_final1 = query_job.to_dataframe()
df_final1.head()

,userID,gender,dob,country,group_verification,method,verified,cost_final,dob_datetime,year_old
0,mpr_2fEpaIsEoNW0DOs3UPIx09sYe1i,M,"4 Mar, 1942, 00:00",SO,B,Sms,0,0.181044,1942-03-04,82
1,mpr_2fKGbFcPcw303XVzowiP6RN3W2t,F,"19 Nov, 1948, 00:00",PK,A,Sms,0,0.180000,1948-11-19,76
2,mpr_2fMhNzBgyiovBWEq9YrrLjgtBMi,M,"22 Dec, 1972, 00:00",PK,B,Sms,1,0.180000,1972-12-22,52
3,mpr_2fEBOB87IHCwNy1Mtp1Y45vqsvu,M,"2 May, 1977, 00:00",PK,A,Sms,0,0.180000,1977-05-02,47
4,mpr_2fKrrSeR9BQayfKEV1ZxUH2KPuz,M,"2 Feb, 1968, 00:00",PK,A,Sms,1,0.180000,1968-02-02,56


In [ ]:
df_final1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18421 entries, 0 to 18420
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   userID              18421 non-null  object        
 1   gender              18421 non-null  object        
 2   dob                 18421 non-null  object        
 3   country             18421 non-null  object        
 4   group_verification  18421 non-null  object        
 5   method              18421 non-null  object        
 6   verified            18421 non-null  Int64         
 7   cost_final          18420 non-null  float64       
 8   dob_datetime        18421 non-null  datetime64[us]
 9   year_old            18421 non-null  Int64         
dtypes: Int64(2), datetime64[us](1), float64(1), object(6)
memory usage: 1.4+ MB


In [ ]:
# Create the 'method2'
df_final1['method2'] = df_final1['method'].apply(lambda x: 0 if x == 'Sms' else 1)

In [ ]:
df_final1.head()

,userID,gender,dob,country,group_verification,method,verified,cost_final,dob_datetime,year_old,method2
0,mpr_2fEpaIsEoNW0DOs3UPIx09sYe1i,M,"4 Mar, 1942, 00:00",SO,B,Sms,0,0.181044,1942-03-04,82,0
1,mpr_2fKGbFcPcw303XVzowiP6RN3W2t,F,"19 Nov, 1948, 00:00",PK,A,Sms,0,0.180000,1948-11-19,76,0
2,mpr_2fMhNzBgyiovBWEq9YrrLjgtBMi,M,"22 Dec, 1972, 00:00",PK,B,Sms,1,0.180000,1972-12-22,52,0
3,mpr_2fEBOB87IHCwNy1Mtp1Y45vqsvu,M,"2 May, 1977, 00:00",PK,A,Sms,0,0.180000,1977-05-02,47,0
4,mpr_2fKrrSeR9BQayfKEV1ZxUH2KPuz,M,"2 Feb, 1968, 00:00",PK,A,Sms,1,0.180000,1968-02-02,56,0


In [ ]:
df_final1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18421 entries, 0 to 18420
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   userID              18421 non-null  object        
 1   gender              18421 non-null  object        
 2   dob                 18421 non-null  object        
 3   country             18421 non-null  object        
 4   group_verification  18421 non-null  object        
 5   method              18421 non-null  object        
 6   verified            18421 non-null  Int64         
 7   cost_final          18420 non-null  float64       
 8   dob_datetime        18421 non-null  datetime64[us]
 9   year_old            18421 non-null  Int64         
 10  method2             18421 non-null  int64         
dtypes: Int64(2), datetime64[us](1), float64(1), int64(1), object(6)
memory usage: 1.6+ MB
